In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn import cross_validation
#from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time 
from sklearn import neighbors
from sklearn.utils import shuffle
tf.logging.set_verbosity(tf.logging.ERROR)
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


C:\Users\saish\Anaconda2\envs\tf\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
mnist = fetch_mldata("MNIST original")

In [3]:
print(mnist.data.shape)

(70000, 784)


In [4]:
print(np.unique(mnist.target))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]


In [5]:
X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X, y = shuffle(X,y)

In [6]:
#X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X_train, y_train = np.float32(X[:15000]), np.float32(y[:15000])
X_test, y_test = np.float32(X[60000:]), np.float32(y[60000:])

In [7]:
# PCA 

In [8]:
pca = PCA(n_components=700)
X_train_pca = pca.fit_transform(X_train, y_train)
X_test_pca = pca.transform(X_test)
print('Variance covered', pca.explained_variance_ratio_.sum())
print('PCA Shape', X_train_pca.shape)

Variance covered 1.0
PCA Shape (15000, 700)


In [9]:
# LDA

In [10]:
lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X_train_pca, y_train)
X_test_lda = lda.transform(X_test_pca)
print('Variance covered', lda.explained_variance_ratio_.sum())
print('PCA Shape', X_train_lda.shape)

Variance covered 1.0
PCA Shape (15000, 9)


In [11]:
start = int(round(time.time() * 1000))

In [12]:
LR = LogisticRegression(solver='newton-cg',max_iter=10,penalty='l2')
LR.fit(X_train_lda,y_train)
end = int(round(time.time() * 1000))
print("--NN fitting finished in ", (end-start), "ms--------------")
predicted=LR.predict(X_test_lda)
expected=y_test

C:\Users\saish\Anaconda2\envs\tf\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


--NN fitting finished in  583 ms--------------


In [13]:
print(cross_validation.cross_val_score(LR, X_train_lda,y_train, cv=5))

C:\Users\saish\Anaconda2\envs\tf\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\saish\Anaconda2\envs\tf\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\saish\Anaconda2\envs\tf\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\saish\Anaconda2\envs\tf\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\saish\Anaconda2\envs\tf\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the

[ 0.89384359  0.8934044   0.886       0.88959306  0.89248748]


In [14]:
print("Classification report for kNN classifier %s:\n%s\n"
     % (LR, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Classification report for kNN classifier LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False):
             precision    recall  f1-score   support

        0.0       0.92      0.96      0.94       959
        1.0       0.89      0.94      0.91      1138
        2.0       0.88      0.79      0.84      1017
        3.0       0.83      0.84      0.84      1009
        4.0       0.81      0.87      0.84       944
        5.0       0.78      0.77      0.77       894
        6.0       0.91      0.91      0.91       981
        7.0       0.84      0.86      0.85      1040
        8.0       0.78      0.77      0.78       954
        9.0       0.83      0.78      0.80      1064

avg / total       0.85      0.85      0.85     10000


Confusion matrix:
[[ 918    0    6    2    2   11    8

In [15]:
print("Accuracy is:",round(metrics.accuracy_score(expected,predicted)*100,2))

Accuracy is: 84.98


In [16]:
print("Test error is:",100-round(metrics.accuracy_score(expected,predicted)*100,2))

Test error is: 15.02
